In [4]:
import os
import sys
import json
import datasets
os.environ['TRANSFORMERS_CACHE'] = '/mnt/swordfish-pool2/milad/hf-cache-new'
os.environ['HF_DATASETS_CACHE'] = '/mnt/swordfish-pool2/milad/hf-cache-new'
os.environ['CUDA_VISIBLE_DEVICES'] = '1,2'
os.environ['OPENAI_API_KEY'] = ''
sys.path.append('./src-py')

In [5]:
os.environ['hf_token'] = "<token>"

In [6]:
import generate_conversations

/local/nlp/milad/conda-envs/trl-library/lib/python3.10/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [7]:
paper_text = """ The Moral Debater:\nA Study on the Computational Generation of Morally Framed Arguments\nMilad Alshomary †\nRoxanne El Baff ‡\nTimon Gurcke †\nHenning Wachsmuth †\n†Paderborn University, Paderborn, Germany, milad.alshomary@upb.de\n‡German Aerospace Center, Oberpfaffenhofen, Germany, Roxanne.ElBaff@dlr.de\nAbstract\nAn audience’s prior beliefs and morals are\nstrong indicators of how likely they will be\naffected by a given argument. Utilizing such\nknowledge can help focus on shared values to\nbring disagreeing parties towards agreement.\nIn argumentation technology, however, this is\nbarely exploited so far. This paper studies the\nfeasibility of automatically generating morally\nframed arguments as well as their effect on dif-\nferent audiences. Following the moral foun-\ndation theory, we propose a system that effec-\ntively generates arguments focusing on differ-\nent morals. In an in-depth user study, we ask\nliberals and conservatives to evaluate the im-\npact of these arguments. Our results suggest\nthat, particularly when prior beliefs are chal-\nlenged, an audience becomes more affected by\nmorally framed arguments.\n1\nIntroduction\nIn the last years, more research has been dedicated\nto studying prior beliefs in argumentation. Un-\nderstanding the role of prior beliefs helps people\ncraft more effective arguments when targeting a\nparticular audience. Accordingly, operationalizing\nknowledge about the audience as part of support-\ning tools for humans or realized in fully automated\ndebating technologies (Slonim et al., 2021) could\nbeneﬁt the production of arguments that bridge the\ngap between disputed parties by focusing on the\nshared beliefs rather than divisive ones (Alshomary\nand Wachsmuth, 2021).\nIn social psychology, a body of research em-\nploys the notion of morals to understand people’s\njudgments on controversial topics (Haidt, 2012;\nFulgoni et al., 2016). Feinberg and Willer (2015)\ndemonstrated that arguments become more effec-\ntive when they match the morals of the target audi-\nence. Multiple works in computational linguistics\nhave analyzed the persuasive effectiveness of argu-\nments depending on the target audience (Durmus\nand Cardie, 2018; El Baff et al., 2020), showing\nthat audience-based features reliably predict effec-\ntiveness. Different proxies of beliefs have been\nproposed as part of this, ranging from interests and\npersonality traits (Al Khatib et al., 2020) to stances\non popular issues (Alshomary et al., 2021a). The\nauthors of the latter used the stances to control\nthe generation of argumentative texts. However,\nthey did not assess the effectiveness of these texts\non the audience, leaving the importance of encod-\ning beliefs ultimately unclear. Beyond that, little\nresearch has been done on generating arguments\ntailored towards a speciﬁc audience, let alone on\nthe importance of morals in achieving agreement.\nThis work studies the feasibility of generating\nmorally framed arguments computationally and the\neffect of these arguments on different audiences.\nFor this, we rely on the moral foundation theory\n(Haidt, 2012) that projects the moral system into\nﬁve foundations: care, fairness, loyalty, authority,\nand purity. To produce arguments that address spe-\nciﬁc morals, we extend the capabilities of Project\nDebater (Slonim et al., 2021).1 Project Debater\nincludes a hybrid approach of multiple components\ndesigned to generate arguments of high quality that\ncompete with human arguments. Building on this\ntechnology helps us focus on evaluating the impact\nof morally framed arguments, as it ensures a certain\nbase quality level in the generation.\nIn particular, our proposed extended system\ntakes as input a controversial topic, a stance, and\na set of morals. It retrieves a set of argumentative\ntexts, ﬁlters the ones conveying the input morals,\nand ﬁnally phrases an argument holding the given\nstance on the given topic focusing on the given\nmorals. To identify morals in texts, we rely on\ndistant supervision: We use the Reddit dataset of\nSchiller et al. (2020), which contains argumenta-\ntive texts with annotated aspects, along with the\n1Available via an API under: https://early-access-program.\ndebater.res.ibm.com/\narXiv:2203.14563v1  [cs.CL]  28 Mar 2022\n\nmoral-to-concept lexicon of Hulpus et al. (2020)\nfor the automatic mapping from aspects to morals.\nThen, we train a BERT-based classiﬁer, achieving\nhigh performance on the moral dataset of Kobbe\net al. (2020) compared to ablation baselines.\nTo assess the effect of morally framed arguments\non a particular audience, we consider liberals and\nconservatives as alternative audiences. We study\nthe question of whether morally loaded arguments\nare more effective on a speciﬁc audience. Addi-\ntionally, we investigate whether differently-framed\nmoral arguments (targeting different morals) vary\nin their effect on liberals and conservatives. In line\nwith El Baff et al. (2018), we design a user study to\nanswer both questions. Our study separately asks\nthree liberals and three conservatives to rank dif-\nferent arguments on speciﬁc controversial issues\nbased on how effective they were in challenging or\nempowering the audience’s stance.\nThe results suggest that, when arguments chal-\nlenge the stance, the morally framed ones are gen-\nerally more effective, especially for the conserva-\ntive audience. We ﬁnd that liberals value argu-\nments that focus on their own morals (care and\nfairness) the most, especially when their stance is\nchallenged. Although conservatives also value re-\nspective arguments, a focus on typically conserva-\ntive morals (loyalty, authority, and purity) becomes\nmore relevant to them when their stance is chal-\nlenged. Despite the limited size of our user study,\nthese ﬁndings hint at the importance of utilizing\nmorals to craft more effective arguments. The code,\nthe trained model, and the data are made publicly\navailable.2 The contributions of this work can be\nsummarized as follows:\n• A state-of-the-art model for mining statements\nwith deﬁned morals from argumentative texts.\n• A system, based on Project Debater, for gen-\nerating morally-framed arguments.\n• A user-study giving empirical evidence for\nthe impact of morally-framed arguments on\naudiences of different political ideologies.\n2\nRelated Work\nPrior Beliefs in Argumentation\nStudying per-\nsuasiveness in argumentation is an active ﬁeld\nof research. Besides planning..."""

In [8]:
sample_dataset = datasets.Dataset.from_list([{"paper_text": paper_text, "paper_title": "The moral debater" }])

In [18]:
generate_conversations.generate_conversations_interactively(sample_dataset, "./test-ds", 
                                     "meta-llama/Meta-Llama-3-8B-Instruct", "Qwen/Qwen2.5-7B-Instruct", 
                                     journalist_adapter_name="/mnt/swordfish-pool2/milad/communicating-science-to-the-public/models/new-qwen-trained-journalist-on-deepseek-3epochs-high-capacity/", 
                                        researcher_adapter_name="", 
                                        baseline_researcher=True,
                                        max_rounds=8)

You are a researcher answering questions about your scientific paper
You are a helpful and knowledgeable journalist asking questions about a scientific paper.
Journalist is meta-llama/Meta-Llama-3-8B-Instruct base with adapter /mnt/swordfish-pool2/milad/communicating-science-to-the-public/models/new-qwen-trained-journalist-on-deepseek-3epochs-high-capacity/
Researcher is Qwen/Qwen2.5-7B-Instruct base with adapter 


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
Device set to use cuda:1
100%|██████████████████████████████████████████████████████████████████████████████████| 8/8 [02:06<00:00, 15.87s/it]


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1 [00:00<?, ? examples/s]

In [19]:
res = datasets.load_from_disk('./test-ds')

In [21]:
print("{}\n---\n\n".format(res['paper_title'][0]),"\n\n==\n\n".join([x['role'] + ': ' + x['content'] for x in res['generated_conversation'][0][2:]]))

The moral debater
---

 assistant: That's interesting. How does the system actually generate these morally-framed arguments?

==

user: The system for generating morally-framed arguments relies on several key components and processes:

1. **Input Processing**: The system takes as input a controversial topic, a specific stance (e.g., for or against), and a set of morals (e.g., care, fairness, loyalty, authority, purity) that the generated arguments should focus on.

2. **Argument Retrieval**: Using the controversial topic and stance, the system retrieves a set of argumentative texts from a large corpus. These texts are relevant to the given topic and stance.

3. **Moral Filtering**: From the retrieved texts, the system filters out those that convey the specified morals. This step involves identifying and selecting texts that align with the desired moral foundations.

4. **Argument Construction**: After filtering, the system constructs a new argument that holds the given stance on the to